In [2]:
import os
import lean_dojo
from lean_dojo import *
from random import randint
import random
import time
import pickle
import ray
import json
import faiss
import subprocess

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

from utils.lean_rag_utils import *

/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-06 18:20:36,274	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
import psutil

# Get the current process object
process = psutil.Process(os.getpid())

# Memory info (in bytes)
mem_info = process.memory_info()

# Print the RSS (Resident Set Size): total physical memory used
print(f"Current memory RSS usage: {mem_info.rss / (1024 ** 2):.2f} MB")

# Print the VMS (Virtual Memory Size): total virtual memory used
print(f"Current memory VMS usage: {mem_info.vms / (1024 ** 2):.2f} MB")

Current memory RSS usage: 605.13 MB
Current memory VMS usage: 4728.60 MB


In [3]:
fin = open('/home/mcwave/code/automath/atp/datasets/tac_templates_in_files/train_tac_templates.json', 'r')
train_tac_templates = json.load(fin)
fin.close()
all_file_paths = list(set([x[-1] for x in train_tac_templates]))

In [7]:
[x for x in train_tac_templates if 'Submodule.eq_top_of_nonempty_interior' in x[2]]

[['rcases hs with ⟨y, hy⟩',
  'rcases {set} with ⟨{nvar0}, {nvar1}⟩',
  "Submodule.eq_top_of_nonempty_interior'",
  '.lake/packages/mathlib/Mathlib/Topology/Algebra/Module/Basic.lean'],
 ["refine' Submodule.eq_top_iff'.2 fun x => _",
  "refine' {custom_def:.topologicalClosure (s : Submodule R M) : Submodule R M}.eq_top_iff'.2 fun x => _",
  "Submodule.eq_top_of_nonempty_interior'",
  '.lake/packages/mathlib/Mathlib/Topology/Algebra/Module/Basic.lean'],
 ['rw [mem_interior_iff_mem_nhds] at hy',
  'rw [mem_interior_iff_mem_nhds] at {hypothesis}',
  "Submodule.eq_top_of_nonempty_interior'",
  '.lake/packages/mathlib/Mathlib/Topology/Algebra/Module/Basic.lean'],
 ['have : Tendsto (fun c : R => y + c • x) (𝓝[{ x : R | IsUnit x }] 0) (𝓝 (y + (0 : R) • x)) :=',
  'have : Tendsto (fun c : {type} => {variable} + c • {variable}) (𝓝[{ {variable} : {type} | IsUnit {variable} }] 0) (𝓝 ({variable} + (0 : {type}) • {variable})) :=',
  "Submodule.eq_top_of_nonempty_interior'",
  '.lake/packages/mathli

In [3]:
# repo = LeanGitRepo(
#     "https://github.com/xiaoxin-yin/math-in-lean",
#     "20077bcd4392317ddb9605404fda3a85e40e8956"
# )

# fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
# traced_repo = pickle.load(fin)
# fin.close()

In [4]:
from utils.lean_rag_utils import *

import math
from lean_dojo import *

# Set the environment variable to disable log deduplication
os.environ['RAY_record_all_task_output'] = '1'
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ensure Ray is not already initialized
if ray.is_initialized():
    ray.shutdown()

# Start Ray with custom configuration
ray.init(
    num_gpus=1,
    num_cpus=1,
    _memory=(192 * 1024 * 1024 * 1024),  # For example, limit Ray to 4 GB of RAM
    object_store_memory=(20 * 1024 * 1024 * 1024)  # Set object store memory to 2 GB
)


@ray.remote(num_cpus=1,num_gpus=0.1)
def compute_provability_training_data_remote(file_path, output_path):
    worker_id = random.randint(0, 100)
    print("Worker", worker_id, "on", file_path)
    cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True).stdout
    print(cuda_version)
    print("CUDA:", torch.cuda.is_available())
    print("os.environ['LD_LIBRARY_PATH']", os.environ['LD_LIBRARY_PATH'])
    print("Worker", worker_id, "Loading repo...")
    repo = LeanGitRepo(
        "https://github.com/xiaoxin-yin/math-in-lean",
        "20077bcd4392317ddb9605404fda3a85e40e8956"
    )
    fin = open('/home/mcwave/code/automath/atp/datasets/train_traced_theorems_repo_math_in_lean.pkl', 'rb')
    train_traced_theorems = pickle.load(fin)
    fin.close()
    print("Worker", worker_id, "Done.")
    #
    print("Worker", worker_id, "Loading models...")
    # Load pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model_state = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_state-batch64-60k-loss0047.model')
    #model_tac   = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_tac-batch64-60k-loss0047.model')
    device = model_state.device
    print(device)
    #
    print("Worker", worker_id, "Loading faiss index ...")
    index = faiss.read_index('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/faiss_index_bert_embeds-batch64-60k-loss0047.idx')
    fin = open('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/tac_template_freq.json', 'r')
    tac_template_freq = json.load(fin)
    fin.close()
    tacs = list(tac_template_freq.keys())
    #
    print("Worker", worker_id, "Computing provability ...")
    compute_provability_training_data(repo,
                                      file_path,
                                      output_path,
                                      train_traced_theorems,
                                      model_state,
                                      tokenizer,
                                      index,
                                      tacs)

# all_file_paths = [
#     "MIL/C02_Basics/solutions/Solutions_S01_Calculating.lean",
#     "MIL/C02_Basics/solutions/Solutions_S02_Proving_Identities_in_Algebraic_Structures.lean",
#     "MIL/C02_Basics/solutions/Solutions_S03_Using_Theorems_and_Lemmas.lean",
#     "MIL/C02_Basics/solutions/Solutions_S04_More_on_Order_and_Divisibility.lean",
#     "MIL/C02_Basics/solutions/Solutions_S05_Proving_Facts_about_Algebraic_Structures.lean"
# ]
output_folder = '/home/mcwave/code/automath/atp/datasets/provability/rag/'

# Submit tasks
result_ids = [compute_provability_training_data_remote.remote(file_path, output_folder) for file_path in all_file_paths]

# Fetch results
results = ray.get(result_ids)

# Optionally, shut down Ray if you're done with all computations
ray.shutdown()

# Display results
print(results)

2024-06-05 00:20:53,747	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(compute_provability_training_data_remote pid=760220) Worker 75 on .lake/packages/mathlib/Mathlib/Data/Set/Intervals/OrdConnected.lean
(compute_provability_training_data_remote pid=760220) nvcc: NVIDIA (R) Cuda compiler driver
(compute_provability_training_data_remote pid=760220) Copyright (c) 2005-2023 NVIDIA Corporation
(compute_provability_training_data_remote pid=760220) Built on Tue_Jun_13_19:16:58_PDT_2023
(compute_provability_training_data_remote pid=760220) Cuda compilation tools, release 12.2, V12.2.91
(compute_provability_training_data_remote pid=760220) Build cuda_12.2.r12.2/compiler.32965470_0
(compute_provability_training_data_remote pid=760220) 
(compute_provability_training_data_remote pid=760220) CUDA: True
(compute_provability_training_data_remote pid=760220) os.environ['LD_LIBRARY_PATH'] /usr/local/cuda-12.2/lib64:
(compute_provability_training_data_remote pid=760220) Worker 75 Loading repo...
(compute_provability_training_data_remote pid=760220) Worker 75 Done.
(comp

(compute_provability_training_data_remote pid=760220) 2024-06-05 00:21:27.046 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=760220) Worker 75 Computing provability ...
(compute_provability_training_data_remote pid=760220) Loading Theorem: Set.ordConnected_of_Ioo theorem ordConnected_of_Ioo {α : Type*} [PartialOrder α] {s : Set α}
(compute_provability_training_data_remote pid=760220)     (hs : ∀ x ∈ s, ∀ y ∈ s, x < y → Ioo x y ⊆ s) : OrdConnected s
(compute_provability_training_data_remote pid=760220) Done
(compute_provability_training_data_remote pid=760220) Sucessfully followed proof
(compute_provability_training_data_remote pid=760220) INIT_STATE: α✝ : Type u_1
(compute_provability_training_data_remote pid=760220) β : Type u_2
(compute_provability_training_data_remote pid=760220) inst✝² : Preorder α✝
(compute_provability_training_data_remote pid=760220) inst✝¹ : Preorder β
(compute_provability_training_data_remote pid=760220) s✝ t : Set α✝
(compute_provability_training_data_remote pid=760220) α : Type u_3
(compute_provability_training_data_remote pid=760220) i

(compute_provability_training_data_remote pid=760220) 2024-06-05 00:28:56.372 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


...

(compute_provability_training_data_remote pid=760220) 2024-06-05 00:37:45.345 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=760220) 1895 states in total
(compute_provability_training_data_remote pid=760220) 1 proven states
(compute_provability_training_data_remote pid=760220) 8 state pairs found in OrderEmbedding.image_Ioc .lake/packages/mathlib/Mathlib/Data/Set/Intervals/OrdConnected.lean
(compute_provability_training_data_remote pid=760220) Loading Theorem: OrderEmbedding.image_Ioo theorem image_Ioo (e : α ↪o β) (he : OrdConnected (range e)) (x y : α) :
(compute_provability_training_data_remote pid=760220)     e '' Ioo x y = Ioo (e x) (e y)
(compute_provability_training_data_remote pid=760220) Done
(compute_provability_training_data_remote pid=760220) Sucessfully followed proof
(compute_provability_training_data_remote pid=760220) INIT_STATE: α : Type u_1
(compute_provability_training_data_remote pid=760220) β : Type u_2
(compute_provability_training_data_remote pid=760220) inst✝¹ : Preorder α
(compute_provability_training_data_remote pid=760220) inst✝ : Preor

(compute_provability_training_data_remote pid=760877) 2024-06-05 01:01:33.136 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=760877) INIT_STATE: f : ℕ → ℚ
(compute_provability_training_data_remote pid=760877) ⊢ IsCauSeq abs f ↔ IsCauSeq abs fun i => ↑(f i)
(compute_provability_training_data_remote pid=760877) {'f': 'function'}
(compute_provability_training_data_remote pid=760877) 10000 steps executed
(compute_provability_training_data_remote pid=760877) 20000 steps executed
(compute_provability_training_data_remote pid=760877) 30000 steps executed
(compute_provability_training_data_remote pid=761062) Worker 74 on .lake/packages/mathlib/Mathlib/MeasureTheory/Measure/AEMeasurable.lean
(compute_provability_training_data_remote pid=761062) nvcc: NVIDIA (R) Cuda compiler driver
(compute_provability_training_data_remote pid=761062) Copyright (c) 2005-2023 NVIDIA Corporation
(compute_provability_training_data_remote pid=761062) Built on Tue_Jun_13_19:16:58_PDT_2023
(compute_provability_training_data_remote pid=761062) Cuda compilation tools, release 12.2, V12.2.91
(comp

OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 192.168.0.192, ID: 671926fa8dfb06c61c26e948eac3754d1d9aea95f8777762687b280c) where the task (task ID: b8f2ebda523be91202ca084ed907e96a3b1bcdc001000000, name=compute_provability_training_data_remote, pid=760877, memory used=92.56GB) was running was 179.16GB / 188.54GB (0.950267), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: 1842cc50fe7cd92eb247ea5ed9968f32318b71c83bc7a0038a0df364) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 192.168.0.192`. To see the logs of the worker, use `ray logs worker-1842cc50fe7cd92eb247ea5ed9968f32318b71c83bc7a0038a0df364*out -ip 192.168.0.192. Top 10 memory users:
PID	MEM(GB)	COMMAND
760877	92.56	ray::compute_provability_training_data_remote
761056	1.88	lean --threads=1 --memory=32768 .lake/packages/mathlib/Mathlib/Data/Real/Archimedean.lean
75

In [5]:
import ray

# Ensure Ray is not already initialized
if ray.is_initialized():
    ray.shutdown()

In [12]:
fin = open('/home/mcwave/code/automath/atp/datasets/tac_templates_in_files/train_tac_templates.json', 'r')
train_tac_templates = json.load(fin)
fin.close()
all_file_paths = list(set([x[-1] for x in train_tac_templates]))

repo = LeanGitRepo(
        "https://github.com/xiaoxin-yin/math-in-lean",
        "20077bcd4392317ddb9605404fda3a85e40e8956"
)

fin = open('/home/mcwave/code/automath/atp/datasets/train_traced_theorems_repo_math_in_lean.pkl', 'rb')
train_traced_theorems = pickle.load(fin)
fin.close()

In [14]:
from utils.lean_rag_utils import *

file_path = '.lake/packages/mathlib/Mathlib/Data/Real/Archimedean.lean'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_state = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_state-batch64-60k-loss0047.model')
#model_tac   = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_tac-batch64-60k-loss0047.model')
device = model_state.device
print(device)
#
index = faiss.read_index('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/faiss_index_bert_embeds-batch64-60k-loss0047.idx')
fin = open('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/tac_template_freq.json', 'r')
tac_template_freq = json.load(fin)
fin.close()
tacs = list(tac_template_freq.keys())
#
output_folder = '/home/mcwave/code/automath/atp/datasets/provability/rag/'
compute_provability_training_data(repo,
                                  file_path,
                                  output_folder,
                                  train_traced_theorems,
                                  model_state,
                                  tokenizer,
                                  index,
                                  tacs)

2024-06-06 18:19:42.292 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


cuda:0
Loading Theorem: Real.isCauSeq_iff_lift theorem isCauSeq_iff_lift {f : ℕ → ℚ} : IsCauSeq abs f ↔ IsCauSeq abs fun i => (f i : ℝ)
Done
INIT_STATE: f : ℕ → ℚ
⊢ IsCauSeq abs f ↔ IsCauSeq abs fun i => ↑(f i)
{'f': 'function'}


SystemExit: -1

/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from utils.lean_math_utils import *

theorem = Theorem(repo, ".lake/packages/mathlib/Mathlib/Data/Nat/Lattice.lean",
                      "Nat.sInf_mem")

dojo, state_0 = Dojo(theorem).__enter__()
print(state_0.pp)

state_1 = dojo.run_tac(state_0, "rw [← mul_assoc]")

#print(state_1)
print(state_1.pp)

state_2 = dojo.run_tac(state_1, "rw [mul_comm a]") 

print(state_2)

dojo.__exit__(None, None, None)

(raylet) [2024-06-05 01:11:53,699 E 759972 759972] (raylet) node_manager.cc:3002: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 671926fa8dfb06c61c26e948eac3754d1d9aea95f8777762687b280c, IP: 192.168.0.192) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.192`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


(compute_provability_training_data_remote pid=761062) Worker 74 Done.
(compute_provability_training_data_remote pid=761062) Worker 74 Loading models...
(compute_provability_training_data_remote pid=761062) cuda:0
(compute_provability_training_data_remote pid=761062) Worker 74 Loading faiss index ...
(compute_provability_training_data_remote pid=761062) Worker 74 Computing provability ...
(compute_provability_training_data_remote pid=761062) Loading Theorem: aemeasurable_of_map_neZero lemma aemeasurable_of_map_neZero {mβ : MeasurableSpace β} {μ : Measure α}
(compute_provability_training_data_remote pid=761062)     {f : α → β} (h : NeZero (μ.map f)) :
(compute_provability_training_data_remote pid=761062)     AEMeasurable f μ := by
(compute_provability_training_data_remote pid=761062)   by_contra h'
(compute_provability_training_data_remote pid=761062)   simp [h'] at h
(compute_provability_training_data_remote pid=761062) Done


(compute_provability_training_data_remote pid=761062) 2024-06-05 01:11:59.218 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=761062) Sucessfully followed proof
(compute_provability_training_data_remote pid=761062) INIT_STATE: ι : Type u_1
(compute_provability_training_data_remote pid=761062) α : Type u_2
(compute_provability_training_data_remote pid=761062) β : Type u_3
(compute_provability_training_data_remote pid=761062) γ : Type u_4
(compute_provability_training_data_remote pid=761062) δ : Type u_5
(compute_provability_training_data_remote pid=761062) R : Type u_6
(compute_provability_training_data_remote pid=761062) m0 : MeasurableSpace α
(compute_provability_training_data_remote pid=761062) inst✝² : MeasurableSpace β
(compute_provability_training_data_remote pid=761062) inst✝¹ : MeasurableSpace γ
(compute_provability_training_data_remote pid=761062) inst✝ : MeasurableSpace δ
(compute_provability_training_data_remote pid=761062) f✝ g : α → β
(compute_provability_training_data_remote pid=761062) μ✝ ν : Measure α
(compute_provability_training_data_remote pid=76

(compute_provability_training_data_remote pid=761062) 2024-06-05 01:14:27.351 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


...

(compute_provability_training_data_remote pid=773180) Sucessfully followed proof
(compute_provability_training_data_remote pid=773180) INIT_STATE: ⊢ True
(compute_provability_training_data_remote pid=773180) {}
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) ProofFinished
(compute_provability_training_data_remote pid=773180) 10 states in total
(compute_provability_training_data_remote pid=773180) 9 proven states
(compute_provability_training_data_remote pid=773180) 9 state pairs found in ghjk .lake/packages/proof

(compute_provability_training_data_remote pid=773358) 2024-06-05 09:04:02.568 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=773358) DojoInitError
(compute_provability_training_data_remote pid=773358) 0 states in total
(compute_provability_training_data_remote pid=773358) 0 proven states
(compute_provability_training_data_remote pid=773358) 0 state pairs found in WithTop.map_sub .lake/packages/mathlib/Mathlib/Algebra/Order/Sub/WithTop.lean
(compute_provability_training_data_remote pid=773537) Worker 26 on .lake/packages/mathlib/Mathlib/Algebra/MonoidAlgebra/Support.lean
(compute_provability_training_data_remote pid=773537) nvcc: NVIDIA (R) Cuda compiler driver
(compute_provability_training_data_remote pid=773537) Copyright (c) 2005-2023 NVIDIA Corporation
(compute_provability_training_data_remote pid=773537) Built on Tue_Jun_13_19:16:58_PDT_2023
(compute_provability_training_data_remote pid=773537) Cuda compilation tools, release 12.2, V12.2.91
(compute_provability_training_data_remote pid=773537) Build cuda_12.2.r12.2/compiler.32965470_0
(compute_provability_tra

(compute_provability_training_data_remote pid=773537) 2024-06-05 09:04:45.628 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


(compute_provability_training_data_remote pid=773537) Worker 26 Computing provability ...
(compute_provability_training_data_remote pid=773537) Loading Theorem: MonoidAlgebra.support_single_mul_subset theorem support_single_mul_subset [DecidableEq G] [Mul G] (f : MonoidAlgebra k G) (r : k) (a : G) :
(compute_provability_training_data_remote pid=773537)     (single a r * f : MonoidAlgebra k G).support ⊆ Finset.image (a * ·) f.support
(compute_provability_training_data_remote pid=773537) Done
(compute_provability_training_data_remote pid=773537) INIT_STATE: k : Type u₁
(compute_provability_training_data_remote pid=773537) G : Type u₂
(compute_provability_training_data_remote pid=773537) inst✝² : Semiring k
(compute_provability_training_data_remote pid=773537) inst✝¹ : DecidableEq G
(compute_provability_training_data_remote pid=773537) inst✝ : Mul G
(compute_provability_training_data_remote pid=773537) f : MonoidAlgebra k G
(compute_provability_training_data_remote pid=773537) r : k
(comp